In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

import io

In [3]:
train = pd.read_csv('data/train_2kmZucJ.csv')
train.shape

(7920, 3)

In [6]:
test = pd.read_csv('data/test_oJQbWVk.csv')
test.shape

(1953, 2)

## Read and inspect data

In [7]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

- 1 represents negative tweet and 0 represents non-negative tweet

In [8]:
train.head().T

,0,1,2,3,4
id,1,2,3,4,5
label,0,0,0,0,1
tweet,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


## Text Processing

In [9]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [10]:
train.head().T

,0,1,2,3,4
id,1,2,3,4,5
label,0,0,0,0,1
tweet,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!
clean_tweet,#fingerprint #Pregnancy Test #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect...,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [11]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [12]:
train.head().T

,0,1,2,3,4
id,1,2,3,4,5
label,0,0,0,0,1
tweet,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!
clean_tweet,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone,finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias…,we love this would you go talk makememories unplug relax iphone smartphone wifi connect...,i'm wired i know i'm george i was made that way iphone cute daventry home,what amazing service apple won't even talk to me about a question i have unless i pay them . for their stupid support


## SpaCy and TensorFlow

In [13]:
# import spaCy's language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [14]:
import tensorflow as tf

In [15]:
import tensorflow_hub as hub

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [16]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

- Training samples x Max length of all strings x Length of ELMo vector
- Every word in the input sentence has an ELMo vector of size 1024.
- To represent the sentence embedding, take the mean of all ELMo vectors of all words in that sentence


In [19]:
def elmo_vectors(x):
    
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [20]:
# To prevent overloading, use batches of size 100 each

list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [ ]:
# Extract ELMo embeddings, this takes a very long time
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

W0829 10:05:26.646033  6368 module_wrapper.py:136] From C:\Users\Sharon\Anaconda3\envs\nlpproj\lib\site-packages\tensorflow_core\python\util\module_wrapper.py:163: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

